Movie Data Cleaning 

In [1]:
# importing the necessary libraries
import pandas as pd
import numpy as np
import zipfile
import os
import sqlite3


# EXTRACT DATABASE FROM ZIP (if not already extracted)

zip_path = r"C:\Users\Administrator\Moringa projects\movie data\movie_group_project-main\movie_group_project-main\zippedData\im.db.zip"
extract_dir = r"C:\Users\Administrator\Moringa projects\movie data\movie_group_project-main\movie_group_project-main\zippedData"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Path of extracted database
db_path = os.path.join(extract_dir, "im.db")


# CONNECT TO SQLITE DATABASE

conn = sqlite3.connect(db_path)


# LOAD THE TWO TABLES INTO DATAFRAMES

movie_basics = pd.read_sql("SELECT * FROM movie_basics", conn)
movie_ratings = pd.read_sql("SELECT * FROM movie_ratings", conn)

# Close database connection
conn.close()


# MERGE (JOIN) BASICS + RATINGS ON movie_id

movies_df = pd.merge(movie_basics, movie_ratings, on="movie_id", how="inner")

# PREVIEW RESULT

print("Merged DataFrame Shape:", movies_df.shape)
print(movies_df.head())

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Administrator\\Moringa projects\\movie data\\movie_group_project-main\\movie_group_project-main\\zippedData\\im.db.zip'

In [ ]:

# LOAD ADDITIONAL DATASETS (Box Office & Budget)


# File paths
gross_path = r"C:\Users\Administrator\Moringa projects\movie data\movie_group_project-main\movie_group_project-main\zippedData\bom.movie_gross.csv.gz"
budget_path = r"C:\Users\Administrator\Moringa projects\movie data\movie_group_project-main\movie_group_project-main\zippedData\tn.movie_budgets.csv.gz"

# Load CSVs (pandas reads .gz directly)

gross_df = pd.read_csv(gross_path, compression='gzip')
budget_df = pd.read_csv(budget_path, compression='gzip')

print("Gross dataset:", gross_df.shape)
print("Budget dataset:", budget_df.shape)





Gross dataset: (3387, 5)
Budget dataset: (5782, 6)


In [ ]:
# CLEANING  COLUMN NAMES FOR MERGING


# Renaming title columns so they match
gross_df.rename(columns={"title": "primary_title"}, inplace=True)
budget_df.rename(columns={"movie": "primary_title"}, inplace=True)

In [ ]:
# CLEANING MONEY COLUMNS (Remove $ and , convert to int)


money_cols = ["production_budget", "domestic_gross", "worldwide_gross"]

for col in money_cols:
    budget_df[col] = budget_df[col].replace('[\$,]', '', regex=True).astype(int)

In [ ]:
#  cleaning Box office  (some values missing or NaN)
gross_df['domestic_gross'] = gross_df['domestic_gross'].replace('[\$,]', '', regex=True)
gross_df['domestic_gross'] = pd.to_numeric(gross_df['domestic_gross'], errors='coerce')

gross_df['foreign_gross'] = gross_df['foreign_gross'].replace('[\$,]', '', regex=True)
gross_df['foreign_gross'] = pd.to_numeric(gross_df['foreign_gross'], errors='coerce')

In [ ]:
# REMOVING DUPLICATES

movies_df = movies_df.drop_duplicates(subset="primary_title")
gross_df = gross_df.drop_duplicates(subset="primary_title")
budget_df = budget_df.drop_duplicates(subset="primary_title")


In [ ]:
# MERGING EVERYTHING INTO ONE DATAFRAME


# 1st merge: IMDB + Gross
merged_df = pd.merge(movies_df, gross_df, on="primary_title", how="left")

# 2nd merge: Add Budget data
merged_df = pd.merge(merged_df, budget_df, on="primary_title", how="left")


In [ ]:
# PREVIEWING FINAL DATA

print("Final Dataset Shape:", merged_df.shape)
print(merged_df.head())
print("\nMissing values per column:\n", merged_df.isnull().sum())

Final Dataset Shape: (69993, 17)
    movie_id                    primary_title              original_title  \
0  tt0063540                        Sunghursh                   Sunghursh   
1  tt0066787  One Day Before the Rainy Season             Ashad Ka Ek Din   
2  tt0069049       The Other Side of the Wind  The Other Side of the Wind   
3  tt0069204                  Sabse Bada Sukh             Sabse Bada Sukh   
4  tt0100275         The Wandering Soap Opera       La Telenovela Errante   

   start_year  runtime_minutes                genres  averagerating  numvotes  \
0        2013            175.0    Action,Crime,Drama            7.0        77   
1        2019            114.0       Biography,Drama            7.2        43   
2        2018            122.0                 Drama            6.9      4517   
3        2018              NaN          Comedy,Drama            6.1        13   
4        2017             80.0  Comedy,Drama,Fantasy            6.5       119   

  studio  domesti

In [ ]:
# taking profits and ROI
merged_df["profit"] = merged_df["worldwide_gross"] - merged_df["production_budget"]
merged_df["ROI"] = (merged_df["profit"] / merged_df["production_budget"]) * 100

In [ ]:
#want to extract yearsonlyif releae_data exists
if "release_date" in merged_df.columns:
    merged_df["release_year"] = pd.to_datetime(merged_df["release_date"], errors="coerce").dt.year
 

In [ ]:
# creating multiple rows per columns
if "genres" in merged_df.columns:
    merged_df["genres"] = merged_df["genres"].str.split(",")
    merged_df = merged_df.explode("genres")

print("✅ Feature engineering complete")

✅ Feature engineering complete


 HANDLING MISSING VALUES
 

In [ ]:
#filling numeric columns with median
num_cols = merged_df.select_dtypes(include=['float64', 'int64']).columns
merged_df[num_cols] = merged_df[num_cols].fillna(merged_df[num_cols].median())


In [ ]:
#filling text columns with unknown
cat_cols = merged_df.select_dtypes(include=['object']).columns
merged_df[cat_cols] = merged_df[cat_cols].fillna("Unknown")

print("✅ Missing values handled")

✅ Missing values handled
